# Ubiquant market prediction - EDA

<img src="https://cdn.pixabay.com/photo/2017/08/30/07/56/clock-2696234_1280.jpg" width="800px">

# Table of contents

- [Motivation](#motivation)
- [Loading packages and data](#packages)
- [Initial explorations (to be continued)](#initial_eda)
- [Feature generation (to be continued)](#new_features)
    - [Operations to perform](#operations)
    - [The feature selector](#selector)
    - [Model structure](#model)
    - [Go, honey, go!](#search)

# Motivation <a class="anchor" id="motivation"></a>

Our goal is to predict a metric (not known by us but related to the return rate) that should help traders to make a trading decision. To solve this task we are given:

* 300 anonymized features
* different investments. They are not the same all the time, there can be different investment ids in the test data than in train [(look here)](https://www.kaggle.com/c/ubiquant-market-prediction/discussion/301693).
* time_ids per investment
* (and row ids)

The features in the training set were derived using real historic data. Furthermore the description says that: 

**the final private leaderboard will be determined using data gathered after the training period closes, which means that the public and private leaderboards will have zero overlap**

What exactly does this mean? Is the test data gathered in temporal order right after the training period? Is it subsequent? Or could it also mean that the test data was gathered later after some time has passed by? For me this is not really clear...

# Loading packages and data <a class="anchor" id="packages"></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.linear_model import Ridge
from scipy.stats import pearsonr

from kaggle_secrets import UserSecretsClient

import wandb
# always wanted to try it out - now it's time to do so! :-) 

import seaborn as sns
sns.set()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
use_wandb=False
if use_wandb:
    user_secrets = UserSecretsClient()
    secret_value_wb = user_secrets.get_secret("wandb")
    ! wandb login $secret_value_wb
    wandb.init(project="ubiquant", name="starter")

In [ ]:
train = pd.read_parquet('../input/ubiquant-parquet/train_low_mem.parquet')
train.head()

In [ ]:
test = pd.read_parquet('../input/ubiquant-parquet/example_test.parquet')
test.head()

# Exploratory Analysis <a class="anchor" id="exploration"></a>

# Initial explorations <a class="anchor" id="initial_eda"></a>

How much samples can be found?

In [ ]:
train.shape[0]

How many investments are present?

In [ ]:
train.investment_id.nunique()

What's the maximum time_id per investment id?

In [ ]:
train.groupby("investment_id").time_id.max().value_counts()

That's interesting! Most of the ids have a max time_id of 1219. But there are a few ids that differ from them. 

In [ ]:
train.groupby("investment_id").time_id.max().value_counts().index.max()

In [ ]:
train.groupby("investment_id").time_id.max().value_counts().index.min()

There are no ids with an id > 1219 or < 62. We should better look at a few examples. By the way - how many outlier samples do we have?

In [ ]:
selection = train.groupby("investment_id").time_id.max()
outlier_inv_ids = selection[selection != 1219].index.values
len(outlier_inv_ids)

In [ ]:
plt.figure(figsize=(20,5))
for n in range(10):
    plt.plot(train[train.investment_id == outlier_inv_ids[n]].time_id,
               train[train.investment_id == outlier_inv_ids[n]].target.cumsum(), '.')
    plt.xlim([0,1220])
    plt.title("Return/target cumsum for outlier investments")
    plt.xlabel("time_id")
    plt.ylabel("cumsum return");

### Insights

* We can clearly see that some investments miss parts of their timeseries or end earlier.
* Looking back into the competition description, we find: "The ID code for an investment. **Not all investment have data in all time IDs**."

### How many investments do we have per time id?

In [ ]:
num_investments_per_time_id = train.groupby("time_id").investment_id.nunique()

plt.figure(figsize=(20,5))
plt.plot(num_investments_per_time_id.index, num_investments_per_time_id.values, 'o', color="black")
plt.xlabel("time_id")
plt.ylabel("count")
plt.title("Number of unique investment_ids given time_id");

### Insights

* The number of unique investment ids seems to be roughly constant in the beginning but has an increasind trend after the crazy id 400. 
* We can see that the number of investments given the time id varies especially around the id 400.

What does that mean...? The description says...

>  The time IDs are in order, but the real time between the time IDs is not constant and will likely be shorter for the final private test set than in the training set.

Hmm... what?!

Does it mean given a single investment the time id is in order but the time between ids can vary (also for this single investment)? If so, do we have different "temporal spaces" per investment? Then time_id 12 for example would be a different time for each investment? Or does it mean that this id 12 is the same real time for all investment ids? 

### How is the target distributed?

In [ ]:
fig, ax = plt.subplots(2,2,figsize=(20,10))
sns.distplot(train.target.values, color="red", ax=ax[0,0])
ax[0,0].set_title("Target distribution")
n = 0
selected_id = train[train.target > 8].investment_id.values[n]
selection = train[train.investment_id == selected_id][["time_id", "target"]]
ax[0,1].set_title("Target distribution of investment id {}".format(selected_id))
sns.distplot(selection.target.values, ax=ax[0,1], color="seagreen")
ax[1,0].plot(selection.time_id.values, selection.target.values)
ax[1,0].set_title("Target timeseries of investment id {}".format(selected_id))
ax[1,1].plot(selection.time_id.values, selection.target.cumsum().values)
ax[1,1].set_title("Target timeseries cumsum of investment id {}".format(selected_id))

### Insights

* Looks like Student's t-distribution (like normal with heavy tails).
* Browsing through different outlier series the heavy tails belong to steep changes given our temporal interval. Looking at the cumsum we can see that it does not necessarily mean that this is a strange behaviour as we could also have strong changes over a small number of time_id steps. 

# How are features (or the target) distributed over time?

# Model structure <a class="anchor" id="model"></a>

Ok, this problem has some difficulties:

* We have investment_ids that can be present in train & test or only in train or only in test. 
* We can have some kind of measurement gaps where values are missing.
* Timeseries can have different lengths in total. 
* The time inverals can vary.



Before adding more and more content about feature engineering, I like to build a loop that will allow to add previous features.

In [ ]:
train.head()

In [ ]:
selection = ["time_id", "investment_id"]
N=100
my_features = ["f_{}".format(n) for n in range(N)]
selection.extend(my_features)

In [ ]:
X = train[selection].copy()
for f in my_features:
    X[f + "_shift1"] = X.groupby("investment_id")[f].shift(1)

    X[f + "_diff"] = X[f] - X[f+"_shift1"]
    X[f + "_diff"] = X[f+"_diff"].fillna(0)

In [ ]:
features = ["f_{}_diff".format(n) for n in range(N)]
features.extend(["f_{}".format(n) for n in range(N)])
print(len(features))

In [ ]:
lm = Ridge()
x_train = X[features]
y_train = train.target
lm.fit(x_train, y_train)

In [ ]:
import ubiquant
env = ubiquant.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test set and sample submission

previous_test_df = train[train.time_id == train.time_id.max()]
for (test_df, sample_prediction_df) in iter_test:
    
    for f in my_features:
        test_df.loc[:, f+"_shift1"] = np.nan
        already_known = previous_test_df[previous_test_df.investment_id.isin(test_df.investment_id)]
        test_df.loc[test_df.investment_id.isin(already_known.investment_id), f+"_shift1"] = already_known[f]
        test_df.loc[:, f+"_diff"] = test_df[f] - test_df[f+"_shift1"]
        test_df.loc[:, f+"_diff"] = test_df.loc[:, f+"_diff"].fillna(0)
    
    pred = lm.predict(test_df[features])
    sample_prediction_df['target'] = pred  # make your predictions here
    env.predict(sample_prediction_df)   # register your predictions
    previous_test_df = test_df.copy()

The loop seems to work. Next topic - feature generation.